In [ ]:
import pandas as pd 
y_data = pd.read_csv("./ESC-50/meta/esc50.csv")
captions = y_data["category"].values


In [ ]:
captions = list(set(captions))
len(captions)

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

In [ ]:
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers", # Using the diffusers-specific ID
    torch_dtype=torch.float16,
   
).to("cuda:0")

In [ ]:
token_embeddings_1 = []
token_embeddings_2 = []
token_embeddings_3 = []

for idx, caption in enumerate(captions):
    with torch.no_grad():
        (_, _, _, _, embed1, embed2, embed3) = pipe.encode_prompt(
            prompt=caption,
            prompt_2=caption,
            prompt_3=caption,
            device=pipe.device,
            num_images_per_prompt=1,
            do_classifier_free_guidance=True
        )

    embed1 = embed1.detach().cpu().numpy()
    embed2 = embed2.detach().cpu().numpy()
    embed3 = embed3.detach().cpu().numpy()

    token_embeddings_1.append(embed1)
    token_embeddings_2.append(embed2)
    token_embeddings_3.append(embed3)

    print(f"\rEmbeddings created upto index :- {idx+1}", end="")


In [ ]:
del pipe

In [ ]:
import gc 
gc.collect()

In [ ]:
import numpy as np
token_embeddings_1 = np.concatenate(token_embeddings_1, axis=0)
token_embeddings_2 = np.concatenate(token_embeddings_2, axis=0)
token_embeddings_3 = np.concatenate(token_embeddings_3, axis=0)

In [ ]:
print(f"Shape of clip_l embeddings :- {token_embeddings_1.shape}")
print(f"Shape of clip-g embeddings :- {token_embeddings_2.shape}")
print(f"Shape of t5_embeddings :- {token_embeddings_3.shape}")

In [ ]:
test = token_embeddings_2[5]
test.shape

In [ ]:
test.shape

In [ ]:
clipl_mse = []
clipl_cosine = []
for i in range(test.shape[0] - 1):
    v1 = test[i]
    v2 = test[i + 1]

    mse = np.mean((v1 - v2) ** 2)
    clipl_mse.append(mse)
len(clipl_mse)

In [ ]:
for i in range(test.shape[0] - 1): 
    v1 = test[i] 
    v2 = test[i+1]
    cosine_similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    clipl_cosine.append(cosine_similarity)


In [ ]:
t5_mse = []


In [ ]:
import math
filtered_data = [v if not math.isinf(v) else 0 for v in clipl_mse]



In [ ]:
filtered_data

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(len(filtered_data)), filtered_data)

